In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
data_set=pd.read_csv("Tweets.csv")
data_set.head(4)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)


In [3]:
data_set=data_set[["airline_sentiment","text"]]
data_set

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [4]:
data_set.isna().sum()

airline_sentiment    0
text                 0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
sentiment_encoder=LabelEncoder()
data_set["airline_sentiment"]=sentiment_encoder.fit_transform(data_set["airline_sentiment"].values)

In [6]:
data_set

,airline_sentiment,text
0,1,@VirginAmerica What @dhepburn said.
1,2,@VirginAmerica plus you've added commercials t...
2,1,@VirginAmerica I didn't today... Must mean I n...
3,0,@VirginAmerica it's really aggressive to blast...
4,0,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,2,@AmericanAir thank you we got on a different f...
14636,0,@AmericanAir leaving over 20 minutes Late Flig...
14637,1,@AmericanAir Please bring American Airlines to...
14638,0,"@AmericanAir you have my money, you change my ..."


In [7]:
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [8]:
corpus=[]

for i in range(len(data_set)):
    sentiment=re.sub("[^a-zA-Z]",' ',data_set["text"][i])
    sentiment=sentiment.lower()
    sentiment=sentiment.split()
    ps=PorterStemmer()
    all_stopwords=stopwords.words("english")
    all_stopwords.remove("not")
    sentiment=[ps.stem(word) for word in sentiment if not word in set(all_stopwords)]
    sentiment=' '.join(sentiment)
    corpus.append(sentiment)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1600)
x=cv.fit_transform(corpus).toarray()
y=data_set.iloc[:,0].values

In [10]:
pickle.dump(cv,open("transform.pkl","wb"))

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05,random_state=42)

## ANN

In [12]:
x_train.shape

(13908, 1600)

In [13]:
from tensorflow.keras.utils import to_categorical

y_train=to_categorical(y_train,len(sentiment_encoder.classes_))
y_test=to_categorical(y_test,len(sentiment_encoder.classes_))

In [14]:
x_test.shape

(732, 1600)

In [15]:
from keras.layers import Dense
from keras.models import Sequential

In [16]:
model=Sequential()
model.add(Dense(units=200,activation="relu",input_shape=(1600,)))
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=50,activation="relu"))
model.add(Dense(units=3,activation="softmax"))

In [54]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=20)

Epoch 1/20
435/435 [==============================] - 2s 4ms/step - loss: 0.5953 - accuracy: 0.7565 - val_loss: 0.4947 - val_accuracy: 0.7937
Epoch 2/20
435/435 [==============================] - 1s 3ms/step - loss: 0.3935 - accuracy: 0.8459 - val_loss: 0.5216 - val_accuracy: 0.7787
Epoch 3/20
435/435 [==============================] - 1s 3ms/step - loss: 0.2554 - accuracy: 0.9040 - val_loss: 0.5866 - val_accuracy: 0.7678
Epoch 4/20
435/435 [==============================] - 2s 4ms/step - loss: 0.1313 - accuracy: 0.9523 - val_loss: 0.7624 - val_accuracy: 0.7814
Epoch 5/20
435/435 [==============================] - 1s 3ms/step - loss: 0.0711 - accuracy: 0.9766 - val_loss: 0.9158 - val_accuracy: 0.7842
Epoch 6/20
435/435 [==============================] - 1s 3ms/step - loss: 0.0495 - accuracy: 0.9830 - val_loss: 1.1134 - val_accuracy: 0.7678
Epoch 7/20
435/435 [==============================] - 1s 3ms/step - loss: 0.0392 - accuracy: 0.9860 - val_loss: 1.2912 - val_accuracy: 0.7773
Epoch 

In [17]:
from keras.models import load_model
model=load_model("sentiment.h5")
test_corpus=[]
text="@VirginAmerica do you miss me? Don't worry we'll be together very soon."

sentiment=re.sub("[^a-zA-Z]",' ',text)
sentiment=sentiment.lower()
sentiment=sentiment.split()
ps=PorterStemmer()
all_stopwords=stopwords.words("english")
all_stopwords.remove("not")
sentiment=[ps.stem(word) for word in sentiment if not word in set(all_stopwords)]
sentiment=' '.join(sentiment)
test_corpus.append(sentiment)

In [18]:
test=cv.transform(test_corpus)
sentiment_encoder.inverse_transform([np.argmax(model.predict(test))])

array(['neutral'], dtype=object)

In [58]:
model.save("sentiment.h5")

In [59]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 200)               320200    
_________________________________________________________________
dense_17 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_18 (Dense)             (None, 50)                6450      
_________________________________________________________________
dense_19 (Dense)             (None, 3)                 153       
Total params: 352,531
Trainable params: 352,531
Non-trainable params: 0
_________________________________________________________________
